In [271]:
from sklearn import tree
import pandas as pd
from sklearn import preprocessing
le = preprocessing.LabelEncoder()


In [272]:
dataset = pd.read_csv("loanstats3.csv")  
dataset['Gender'] = dataset['Gender'].fillna( dataset['Gender'].dropna().mode().values[0] )
dataset['Married'] = dataset['Married'].fillna( dataset['Married'].dropna().mode().values[0] )
dataset['Dependents'] = dataset['Dependents'].fillna( dataset['Dependents'].dropna().mode().values[0] )
dataset['Self_Employed'] = dataset['Self_Employed'].fillna( dataset['Self_Employed'].dropna().mode().values[0] )
dataset['LoanAmount'] = dataset['LoanAmount'].fillna( dataset['LoanAmount'].dropna().mean() )
dataset['Loan_Amount_Term'] = dataset['Loan_Amount_Term'].fillna( dataset['Loan_Amount_Term'].dropna().mode().values[0] )
dataset['Credit_History'] = dataset['Credit_History'].fillna( dataset['Credit_History'].dropna().mode().values[0] )
dataset['Dependents'] = dataset['Dependents'].str.rstrip('+')
dataset['Gender'] = dataset['Gender'].map({'Female':0,'Male':1}).astype(np.int)
dataset['Married'] = dataset['Married'].map({'No':0, 'Yes':1}).astype(np.int)
dataset['Education'] = dataset['Education'].map({'Not Graduate':0, 'Graduate':1}).astype(np.int)
dataset['Self_Employed'] = dataset['Self_Employed'].map({'No':0, 'Yes':1}).astype(np.int)
dataset['Loan_Status'] = dataset['Loan_Status'].map({'N':0, 'Y':1}).astype(np.int)
dataset['Dependents'] = dataset['Dependents'].astype(np.int)
for column_name in dataset.columns:
        if dataset[column_name].dtype == object:
            dataset[column_name] = le.fit_transform(dataset[column_name])
        else:
            pass

In [273]:
dataset.shape  

(981, 13)

In [274]:
dataset.head()


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,0,1,0,0,1,0,5849,0.0,142.51153,360.0,1.0,2,1
1,1,1,1,1,1,0,4583,1508.0,128.00000,360.0,1.0,0,0
2,2,1,1,0,1,1,3000,0.0,66.00000,360.0,1.0,2,1
3,3,1,1,0,0,0,2583,2358.0,120.00000,360.0,1.0,2,1
4,4,1,0,0,1,0,6000,0.0,141.00000,360.0,1.0,2,1


In [275]:
X = dataset.drop('Loan_Status', axis=1)  
y = dataset['Loan_Status']  

In [276]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [277]:
from sklearn.tree import DecisionTreeClassifier  
classifier = DecisionTreeClassifier()  
classifier.fit(X_train, y_train)  

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [278]:
y_pred = classifier.predict(X_test)  

In [279]:
from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred))  

[[ 29  22]
 [ 16 130]]
             precision    recall  f1-score   support

          0       0.64      0.57      0.60        51
          1       0.86      0.89      0.87       146

avg / total       0.80      0.81      0.80       197



In [280]:
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
# random forest model creation
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)
# predictions
rfc_predict = rfc.predict(X_test)

In [281]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

In [282]:
rfc_cv_score = cross_val_score(rfc, X, y, cv=10, scoring='roc_auc')

In [283]:
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, rfc_predict))
print('\n')

=== Confusion Matrix ===
[[ 30  21]
 [  7 139]]


=== Classification Report ===
             precision    recall  f1-score   support

          0       0.81      0.59      0.68        51
          1       0.87      0.95      0.91       146

avg / total       0.85      0.86      0.85       197





In [284]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [285]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [286]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [287]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [288]:
preds = xg_reg.predict(X_test)

In [289]:
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, preds.round()))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, preds.round()))
print('\n')

=== Confusion Matrix ===
[[  6  45]
 [  0 146]]


=== Classification Report ===
             precision    recall  f1-score   support

          0       1.00      0.12      0.21        51
          1       0.76      1.00      0.87       146

avg / total       0.83      0.77      0.70       197



